In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.12:3.4.0 --conf spark.cassandra.connection.host=192.168.112.4 pyspark-shell'

In [2]:
import json
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, FloatType, ArrayType
from pyspark.sql.functions import col, explode
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.master('local[12]').appName("btc_insertion").getOrCreate()

23/08/12 15:35:04 WARN Utils: Your hostname, Obi-Wan resolves to a loopback address: 127.0.1.1; using 192.168.1.3 instead (on interface enp6s0)
23/08/12 15:35:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/narvy/.ivy2/cache
The jars for the packages stored in: /home/narvy/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e6346d2b-8f5a-47bd-895e-1133c1460a09;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central


:: loading settings :: url = jar:file:/home/narvy/Avatarify/miniconda3/envs/data-infra/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.github.spotbugs#spotbugs-annotations;3.1.12 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found com.datastax.oss#java-driver-mapper-runtime;4.13.0 in central
	found com.datastax.oss#java-driver-query-builder;4.13.0 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org

In [3]:
input_schema = StructType([
        StructField("ot", ArrayType(FloatType())),
        StructField("ct", ArrayType(FloatType())),
        StructField("o", ArrayType(FloatType())),
        StructField("h", ArrayType(FloatType())),
        StructField("l", ArrayType(FloatType())),
        StructField("c", ArrayType(FloatType())),
        StructField("v", ArrayType(FloatType()))
])

data = json.load(open('../data/dataset.json', 'r'))

In [4]:
conf = SparkConf().setAppName("df_cassandra")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

batch_size = 500
current_index = 0

for interval in data:
    data_i = data[interval]

    while len(data_i['ot'][current_index:batch_size]) > 0:
    
        df = spark.createDataFrame([Row(
            ot = data_i['ot'][current_index:batch_size],
            ct = data_i['ct'][current_index:batch_size],
            o = data_i['o'][current_index:batch_size],
            h = data_i['h'][current_index:batch_size],
            l = data_i['l'][current_index:batch_size],
            c = data_i['c'][current_index:batch_size],
            v = data_i['v'][current_index:batch_size]
        )], schema=input_schema)

        current_index = data_i['ot'].index(data_i['ot'][current_index:batch_size][-1])
    
        df = df\
            .withColumn('ot', explode(df['ot']))\
            .withColumn('ct', explode(df['ct']))\
            .withColumn('o', explode(df['o']))\
            .withColumn('h', explode(df['h']))\
            .withColumn('l', explode(df['l']))\
            .withColumn('c', explode(df['c']))\
            .withColumn('v', explode(df['v']))
    
        df = df.withColumn('ot', col('ot').cast("Timestamp")).withColumn('ct', col('ct').cast("Timestamp"))
    
        df.write\
            .format("org.apache.spark.sql.cassandra")\
            .options(table = f'raw_{interval}', keyspace = 'bitcoin_chart')\
            .mode('append')\
            .save()
        
        del df
        del data_i
    

Exception in thread "refresh progress" Exception in thread "RemoteBlock-temp-file-clean-thread" Exception in thread "Spark Context Cleaner" java.lang.OutOfMemoryError: Java heap space
java.lang.OutOfMemoryError: Java heap space
Exception in thread "executor-heartbeater" java.lang.OutOfMemoryError: Java heap space
java.lang.OutOfMemoryError: Java heap space
Exception in thread "driver-heartbeater" java.lang.OutOfMemoryError: Java heap space
23/08/12 15:35:49 ERROR Executor: Exception in task 11.0 in stage 0.0 (TID 11)
java.lang.OutOfMemoryError: Java heap space
23/08/12 15:35:49 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 11.0 in stage 0.0 (TID 11),5,main]
java.lang.OutOfMemoryError: Java heap space
23/08/12 15:35:49 WARN TaskSetManager: Lost task 11.0 in stage 0.0 (TID 11) (192.168.1.3 executor driver): java.lang.OutOfMemoryError: Java heap space

23/08/12 15:35:49 ERROR TaskSetManager: Task 11 in stage 0.0 failed 1 time

ConnectionRefusedError: [Errno 111] Connection refused